<h2>Capstone Project - The Battle of Neighborhoods </h2> <br>
*by Stephan Bartelheim*

**1 Introduction**  
Düsseldorf is a german city with the highest concentration of japanese citizens outside Japan. Apart from a big number of subsidiaries of big Japanese companies and banks, there is also a plethora of japanese shops and restaurants. Our client a recent arrival from Japan wishes to open another restaurant in the city. Since he is unfamiliar with the place, he needs advice on where to open and what kind of food to offer. We, therefore, try to identify areas in which japanese restaurants tend to thrive and then pick one where the competition is particularly weak. Lastly we will choose a category of japanese food (e.g. Ramen, Sushi, general) that is generally popular but again faces weak competition in our selected area. In the end the client will be left with a very specific recommendation.

**2 Data**  
The area data can be accessed under https://www.dasoertliche.de/Themen/Postleitzahlen/D%C3%BCsseldorf.html. It is from the german phone register provided by the *Deutsche Telekom AG* and is free to use. Unlike in the previous exercises we will divide the city by postal codes and not by neighborhoods, since that is how the data is structured. We will use the *geocoder package* to add coordinates and the FourSquare API *explore* call to retrieve data about the composition of the areas. This will give us a list of venues listed on FourSquare. Apart from business this can for example be tourist sites or sites for access to public transport. Lastly the data about the competing venues in the chosen area is retrieved from the FourSqare API to see how those businesses are rated by customers.

**3 Methodology**

In [1]:
%%capture
#install and import required packages

!pip install lxml beautifulsoup4 html5lib matplotlib -U
import pandas as pd
import numpy as np

import requests
import json

!pip install geocoder
import geocoder

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install folium
import folium 


In [2]:
# We scrape the neighborhood data from the phone register
list_of_dfs = pd.read_html('https://www.dasoertliche.de/Themen/Postleitzahlen/Duesseldorf.html')

Looking at the data we see that there is 38 areas. There is several redundant columns and the and the column labels are in german. We right these problems and add coordinates using the geocoder package with the arcgis API.

In [3]:
DUS=list_of_dfs[0]
DUS

,PLZ,Ortsname,Ortsteil,Landkreis,Bundesland
0,40210,Düsseldorf,FriedrichstadtStadtmitte,Stadt Düsseldorf,Nordrhein-Westfalen
1,40211,Düsseldorf,PempelfortStadtmitte,Stadt Düsseldorf,Nordrhein-Westfalen
2,40212,Düsseldorf,FriedrichstadtStadtmitte,Stadt Düsseldorf,Nordrhein-Westfalen
3,40213,Düsseldorf,AltstadtCarlstadtFriedrichstadtPempelfortStadt...,Stadt Düsseldorf,Nordrhein-Westfalen
4,40215,Düsseldorf,Friedrichstadt,Stadt Düsseldorf,Nordrhein-Westfalen
5,40217,Düsseldorf,FriedrichstadtUnterbilk,Stadt Düsseldorf,Nordrhein-Westfalen
6,40219,Düsseldorf,HafenUnterbilk,Stadt Düsseldorf,Nordrhein-Westfalen
7,40221,Düsseldorf,BilkFleheHafenHammUnterbilkVolmerswerth,Stadt Düsseldorf,Nordrhein-Westfalen
8,40223,Düsseldorf,BilkFleheUnterbilk,Stadt Düsseldorf,Nordrhein-Westfalen
9,40225,Düsseldorf,BilkFleheOberbilkWersten,Stadt Düsseldorf,Nordrhein-Westfalen


In [4]:
# drop  useless column and translate column names
DUS.drop(columns=['Landkreis','Bundesland','Ortsname'],inplace=True)
DUS.rename(columns={"PLZ":"Postal Code","Ortsteil":"Neighborhood"},inplace=True)

In [5]:
# adding coordinates
latitude=[]
longitude=[]
for code in DUS['Postal Code']:
    g = geocoder.arcgis('{}, Düsseldorf, Germany'.format(code))
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Düsseldorf, Germany'.format(code))
    latitude.append(g.latlng[0])
    longitude.append(g.latlng[1])
    
DUS['Latitude'],DUS['Longitude']=latitude, longitude

Since some neighborhoods contain several postal codes, we will perform the further analysis based on the unique postal codes instead of the neighborhoods as in the Toronto and Manhattan analyses. Next we use the FourSqaure API to retrieve lists of venues from our areas.

In [6]:
# The code was removed by Watson Studio for sharing.

In [7]:
# define function to use the FourSquare API explore call
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue_ID',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
%%capture
# retrieve venues for postcode areas
dusseldorf_venues = getNearbyVenues(names=DUS['Postal Code'],
                                   latitudes=DUS['Latitude'],
                                   longitudes=DUS['Longitude']
                                  )

Looking at the number of venues we were able to retrieve by neighborhood we see that there is some areas with very few venues. This could mean that there are really very few businesses in these areas in which case it's probably not an area where a restaurant could strive or that we are just lacking data in which case we can't say much about the characteristics of the area. Here more data would be required. For now we exclude areas with less than 15 venues.

In [9]:
#numbers of venues per area
dusseldorf_venues.groupby('Postal Code').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue_ID,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,,
40210,100,100,100,100,100,100,100
40211,38,38,38,38,38,38,38
40212,100,100,100,100,100,100,100
40213,100,100,100,100,100,100,100
40215,64,64,64,64,64,64,64
40217,70,70,70,70,70,70,70
40219,79,79,79,79,79,79,79
40221,2,2,2,2,2,2,2
40223,21,21,21,21,21,21,21


In [10]:
# filter out areas with sparse data
dus_venues_red=dusseldorf_venues.groupby('Postal Code').filter(lambda x:len(x)>15)

Next we see that there are 850 venues left in total coming from 167 categories. This would later result in very high variance model. We therefore group similar categories together and remove the remaining that appear less than 3 times. In the process we only lose 25 rows and reduce the number of categories to 14.

In [11]:
dus_venues_red.shape

(850, 8)

In [12]:
# number of venue categories
len(dus_venues_red['Venue Category'].unique())

167

In [13]:
# list of venue categories
dus_venues_red['Venue Category'].unique()

array(['Grocery Store', 'Japanese Restaurant', 'Ramen Restaurant',
       'Italian Restaurant', 'Korean Restaurant', 'Arts & Crafts Store',
       'Pastry Shop', 'Souvlaki Shop', 'Ice Cream Shop', 'Café',
       'Chinese Restaurant', 'Vegetarian / Vegan Restaurant', 'Bar',
       'Hotel', 'Cocktail Bar', 'Sushi Restaurant', 'Brewery',
       'Burger Joint', 'Bakery', 'Ethiopian Restaurant',
       'Thai Restaurant', 'Greek Restaurant', 'Bubble Tea Shop',
       'Frozen Yogurt Shop', 'Breakfast Spot', 'Turkish Restaurant',
       'Indie Movie Theater', 'Soba Restaurant', 'Bookstore', 'Theater',
       'North Indian Restaurant', 'Coffee Shop', 'Cigkofte Place',
       'Seafood Restaurant', 'Bistro', 'Doner Restaurant', 'Salad Place',
       'General Entertainment', 'Fast Food Restaurant', 'Pizza Place',
       'Pharmacy', 'Donut Shop', 'Advertising Agency',
       'Mexican Restaurant', 'Middle Eastern Restaurant',
       'Sporting Goods Shop', 'Shoe Store', 'Trattoria/Osteria',
       'Y

In [14]:
# duplicate category column for simplification
dus_venues_red['Category simple']=dus_venues_red['Venue Category']

In [15]:
#grouping of categories for simplified categories
dus_venues_red['Category simple'].replace(regex=['^Ramen.*','^Sushi.*','^Japanese.*','^Soba.*'] , value='Japanese',inplace=True)
dus_venues_red['Category simple'].replace(regex=['.*[Ss]hop.*','.*[Ss]tore.*','.*Boutique.*'] , value='Shopping',inplace=True)
dus_venues_red['Category simple'].replace(regex=['.*Restaurant.*','^Pizza.*','^Burger.*','^Bistro.*','^Steak.*','^Trattoria.*','^BBQ.*','^Deli.*','^Sandwich.*','^Breakfast.*','^Soup.*'], value='Restaurant other',inplace=True)
dus_venues_red['Category simple'].replace(regex=['.*Bar.*','.*[Pp]ub.*','^Taverna.*','^Nightclub','^Brewery.*','^Rock.*'], value='Drinking Place',inplace=True)
dus_venues_red['Category simple'].replace(regex=['.*[Gg]ym.*','.*[Ss]occer.*','.*[Ss]port.*','.*Yoga.*','.*Hockey.*'], value='Sports Venue',inplace=True)
dus_venues_red['Category simple'].replace(regex=['.*[Mm]arket.*','.*[Gg]rocer.*'], value='Groceries',inplace=True)
dus_venues_red['Category simple'].replace(regex=['.*[Mm]useum.*','.*[Tt]heater.*','.*[Gg]allery.*','.*Site.*','^Opera.*'], value='Culture',inplace=True)
dus_venues_red['Category simple'].replace(regex=['.*[Ss]top.*','.*[Ss]tation.*','[Pp]latform.*'], value='Public Transport',inplace=True)
dus_venues_red['Category simple'].replace(regex=['.*Hostel.*'], value='Hotel',inplace=True)
dus_venues_red['Category simple'].replace(regex=['.*[Pp]ark.*','.*Playground.*','.*[Pp]laza.*','^Fountain.*'], value='Recreation',inplace=True)
dus_venues_red['Category simple'].replace(regex=['.*[Aa]gency.*','.*[Ss]ervice.*'], value='Business Services',inplace=True)
len(dus_venues_red['Category simple'].unique())

37

In [16]:
# filter out venues from rare categories
dus_venues_red=dus_venues_red.groupby('Category simple').filter(lambda x:len(x)>2)

In [17]:
dus_venues_red.shape

(825, 9)

In [18]:
# frequency of new categories
dus_venues_red.groupby('Category simple')['Venue'].count()

Category simple
Bakery                25
Bank                   5
Business Services      5
Café                  51
Culture               17
Drinking Place        72
Groceries             24
Hotel                 55
Japanese              44
Public Transport      19
Recreation            28
Restaurant other     295
Shopping             174
Sports Venue          11
Name: Venue, dtype: int64

We next create dummy variables from the simplified categories, calculate mean value for the areas and perform knn clustering. We do not, however, use the 'Japanese' dummy since we want to know what characterises areas with lots of japanese restaruants apart from the fact that there is a lot of japanese restaurants. Working with 7 clusters here gives the nicest result with relatively balanced clusters and a geographical pattern where areas on the same side of the city tend to fall into the same cluster and central and outside areas as well.

In [19]:
# creating dummies for clustering

# one hot encoding
dus_onehot = pd.get_dummies(dus_venues_red[['Category simple']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dus_onehot.insert(loc=0,column='Postal Code', value=dus_venues_red['Postal Code'])

dus_grouped = dus_onehot.groupby('Postal Code').mean().reset_index()
dus_grouped

,Postal Code,Bakery,Bank,Business Services,Café,Culture,Drinking Place,Groceries,Hotel,Japanese,Public Transport,Recreation,Restaurant other,Shopping,Sports Venue
0,40210,0.020833,0.000000,0.000000,0.052083,0.041667,0.041667,0.000000,0.093750,0.229167,0.000000,0.000000,0.364583,0.156250,0.000000
1,40211,0.027027,0.000000,0.027027,0.081081,0.027027,0.054054,0.000000,0.162162,0.054054,0.027027,0.027027,0.405405,0.081081,0.027027
2,40212,0.020202,0.000000,0.000000,0.040404,0.030303,0.030303,0.000000,0.070707,0.141414,0.000000,0.040404,0.161616,0.464646,0.000000
3,40213,0.020833,0.000000,0.000000,0.062500,0.052083,0.187500,0.020833,0.031250,0.000000,0.000000,0.072917,0.270833,0.281250,0.000000
4,40215,0.063492,0.000000,0.031746,0.111111,0.015873,0.126984,0.000000,0.174603,0.015873,0.047619,0.015873,0.285714,0.111111,0.000000
5,40217,0.014925,0.000000,0.000000,0.059701,0.014925,0.074627,0.059701,0.044776,0.014925,0.014925,0.059701,0.402985,0.208955,0.029851
6,40219,0.026667,0.013333,0.000000,0.106667,0.000000,0.080000,0.013333,0.000000,0.013333,0.053333,0.040000,0.520000,0.133333,0.000000
7,40223,0.100000,0.000000,0.000000,0.000000,0.050000,0.000000,0.150000,0.050000,0.000000,0.000000,0.050000,0.150000,0.350000,0.100000
8,40227,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.040000,0.120000,0.000000,0.080000,0.000000,0.360000,0.280000,0.040000
9,40233,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.041667,0.083333,0.000000,0.000000,0.041667,0.416667,0.250000,0.083333


In [20]:
# set number of clusters
kclusters = 7

dus_grouped_clustering = dus_grouped.drop(['Postal Code','Japanese'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dus_grouped_clustering)
kmeans.labels_[0:16]

array([4, 4, 1, 3, 4, 6, 2, 1, 6, 6, 2, 2, 2, 0, 5], dtype=int32)

In [21]:
# add clustering labels
dus_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

dus_merged = DUS

# merge labels on original dataframe
dus_merged = dus_merged.join(dus_grouped.set_index('Postal Code'), on='Postal Code',how='inner')

In [22]:
# create map
map_clusters = folium.Map(location=[51.2277, 6.7735], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

rainbow

['#8000ff', '#2c7ef7', '#2adddd', '#80ffb4', '#d4dd80', '#ff7e41', '#ff0000']

In [23]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dus_merged['Latitude'], dus_merged['Longitude'], dus_merged['Postal Code'], dus_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [24]:
dus_merged

,Postal Code,Neighborhood,Latitude,Longitude,Cluster Labels,Bakery,Bank,Business Services,Café,Culture,Drinking Place,Groceries,Hotel,Japanese,Public Transport,Recreation,Restaurant other,Shopping,Sports Venue
0,40210,FriedrichstadtStadtmitte,51.221500,6.789191,4,0.020833,0.000000,0.000000,0.052083,0.041667,0.041667,0.000000,0.093750,0.229167,0.000000,0.000000,0.364583,0.156250,0.000000
1,40211,PempelfortStadtmitte,51.229510,6.789159,4,0.027027,0.000000,0.027027,0.081081,0.027027,0.054054,0.000000,0.162162,0.054054,0.027027,0.027027,0.405405,0.081081,0.027027
2,40212,FriedrichstadtStadtmitte,51.223825,6.782230,1,0.020202,0.000000,0.000000,0.040404,0.030303,0.030303,0.000000,0.070707,0.141414,0.000000,0.040404,0.161616,0.464646,0.000000
3,40213,AltstadtCarlstadtFriedrichstadtPempelfortStadt...,51.224287,6.773790,3,0.020833,0.000000,0.000000,0.062500,0.052083,0.187500,0.020833,0.031250,0.000000,0.000000,0.072917,0.270833,0.281250,0.000000
4,40215,Friedrichstadt,51.213835,6.784225,4,0.063492,0.000000,0.031746,0.111111,0.015873,0.126984,0.000000,0.174603,0.015873,0.047619,0.015873,0.285714,0.111111,0.000000
5,40217,FriedrichstadtUnterbilk,51.212566,6.774075,6,0.014925,0.000000,0.000000,0.059701,0.014925,0.074627,0.059701,0.044776,0.014925,0.014925,0.059701,0.402985,0.208955,0.029851
6,40219,HafenUnterbilk,51.214015,6.762607,2,0.026667,0.013333,0.000000,0.106667,0.000000,0.080000,0.013333,0.000000,0.013333,0.053333,0.040000,0.520000,0.133333,0.000000
8,40223,BilkFleheUnterbilk,51.199995,6.771755,1,0.100000,0.000000,0.000000,0.000000,0.050000,0.000000,0.150000,0.050000,0.000000,0.000000,0.050000,0.150000,0.350000,0.100000
10,40227,EllerOberbilk,51.213380,6.801196,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.040000,0.120000,0.000000,0.080000,0.000000,0.360000,0.280000,0.040000
13,40233,Flingern NordFlingern SüdLierenfeldStadtmitte,51.221545,6.812467,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.041667,0.083333,0.000000,0.000000,0.041667,0.416667,0.250000,0.083333


We see that the cluster 4 located center-east has the highest share of japanese restaurants (10% of all venues). So this seems be an environment where japanese restaurants do thrive.

In [25]:
#mean values for clusters
dus_merged.groupby('Cluster Labels').mean()

,Postal Code,Latitude,Longitude,Bakery,Bank,Business Services,Café,Culture,Drinking Place,Groceries,Hotel,Japanese,Public Transport,Recreation,Restaurant other,Shopping,Sports Venue
Cluster Labels,,,,,,,,,,,,,,,,,
0,40545.000000,51.229753,6.751326,0.034483,0.034483,0.000000,0.068966,0.000000,0.275862,0.034483,0.034483,0.000000,0.034483,0.068966,0.344828,0.034483,0.034483
1,40217.500000,51.211910,6.776992,0.060101,0.000000,0.000000,0.020202,0.040152,0.015152,0.075000,0.060354,0.070707,0.000000,0.045202,0.155808,0.407323,0.050000
2,40412.750000,51.233663,6.778073,0.043940,0.007644,0.000000,0.054421,0.004808,0.078203,0.036296,0.043974,0.018157,0.028958,0.029134,0.513123,0.141341,0.000000
3,40213.000000,51.224287,6.773790,0.020833,0.000000,0.000000,0.062500,0.052083,0.187500,0.020833,0.031250,0.000000,0.000000,0.072917,0.270833,0.281250,0.000000
4,40212.000000,51.221615,6.787525,0.037117,0.000000,0.019591,0.081425,0.028189,0.074235,0.000000,0.143505,0.099698,0.024882,0.014300,0.351901,0.116147,0.009009
5,40597.000000,51.165470,6.874402,0.000000,0.055556,0.055556,0.166667,0.000000,0.055556,0.111111,0.000000,0.000000,0.111111,0.000000,0.166667,0.222222,0.055556
6,40225.666667,51.215830,6.795913,0.004975,0.000000,0.000000,0.019900,0.004975,0.079320,0.047123,0.082703,0.004975,0.031642,0.033789,0.393217,0.246318,0.051061


Obviously we do want as little competition as possible for our client as well. So we next check in which neighborhood within this promisiong cluster the share is the lowest. In cluster 4 only 1.59% of venues are japanese restaurants.

In [26]:
#postal code areas in the cluster
dus_merged[dus_merged['Cluster Labels']==4]

,Postal Code,Neighborhood,Latitude,Longitude,Cluster Labels,Bakery,Bank,Business Services,Café,Culture,Drinking Place,Groceries,Hotel,Japanese,Public Transport,Recreation,Restaurant other,Shopping,Sports Venue
0,40210,FriedrichstadtStadtmitte,51.221500,6.789191,4,0.020833,0.0,0.000000,0.052083,0.041667,0.041667,0.0,0.093750,0.229167,0.000000,0.000000,0.364583,0.156250,0.000000
1,40211,PempelfortStadtmitte,51.229510,6.789159,4,0.027027,0.0,0.027027,0.081081,0.027027,0.054054,0.0,0.162162,0.054054,0.027027,0.027027,0.405405,0.081081,0.027027
4,40215,Friedrichstadt,51.213835,6.784225,4,0.063492,0.0,0.031746,0.111111,0.015873,0.126984,0.0,0.174603,0.015873,0.047619,0.015873,0.285714,0.111111,0.000000


Looking in detail we see that there is in fact only one japanese restaurant in that area, the Tokyo Lounge, listed as a general japanese restaurant with a not outstanding rating of 7.6. We also see that there is a lot of Ramen restaurants in the other areas, so they appear to be quite popular.

In [27]:
#japanese restaurants in the clusters
dus_venues_red[(dus_venues_red['Postal Code'].isin([40215,40211,40210])) & (dus_venues_red['Category simple']=='Japanese')]

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue_ID,Venue,Venue Latitude,Venue Longitude,Venue Category,Category simple
1,40210,51.221500,6.789191,4b3b8db9f964a520b27525e3,Kushi Tei of Tokyo,51.223275,6.789558,Japanese Restaurant,Japanese
2,40210,51.221500,6.789191,4b448154f964a520bef525e3,Kagaya,51.221320,6.788232,Japanese Restaurant,Japanese
3,40210,51.221500,6.789191,5053696ce4b08e1d3c985b79,Nagomi,51.221913,6.786502,Japanese Restaurant,Japanese
4,40210,51.221500,6.789191,4b3be8f6f964a520207e25e3,Takumi,51.223429,6.788531,Ramen Restaurant,Japanese
21,40210,51.221500,6.789191,4b7e729cf964a52072ed2fe3,Hyuga,51.224525,6.789297,Sushi Restaurant,Japanese
24,40210,51.221500,6.789191,4f115acee4b09e81d8909f8a,Waraku,51.223664,6.787536,Japanese Restaurant,Japanese
26,40210,51.221500,6.789191,53429155498e8dd5982f0e8e,Takezo Ramen Bar,51.222617,6.790507,Ramen Restaurant,Japanese
27,40210,51.221500,6.789191,5b0c5e3a31ac6c002c6c3868,Tonkatsu Gonta,51.223490,6.788785,Japanese Restaurant,Japanese
32,40210,51.221500,6.789191,4bb64cb06edc76b092b7301c,Naniwa,51.224915,6.788172,Ramen Restaurant,Japanese
34,40210,51.221500,6.789191,59da041146e1b64f7c9cbe9f,Takumi 3rd Tori & Veggie,51.224532,6.788735,Ramen Restaurant,Japanese


In [28]:
#competitor ID

competitorID=dus_venues_red[(dus_venues_red['Postal Code']==40215) & (dus_venues_red['Category simple']=='Japanese')].Venue_ID.values[0]

In [29]:
#retrieve competitor information from FourSquare

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(competitorID,CLIENT_ID,CLIENT_SECRET,VERSION)
         
competitor = requests.get(url).json()

In [30]:
#display the average rating

print(competitor['response']['venue']['rating'])

7.6


**5 Results**  
We have identified a cluster of three similarly structured adjacent post code areas in which japaese restaurants appear widely popular with an average density of japanese restaurants of almost 10% of all listed venues. Within this cluster, however, we made out one cluster with only one japanese restaurant, offering unspecified japanese cuisine, with an average rating 7.6. This seems to be a very promising area to open another restaurant. We further discovered that Ramen restaurants are the next common after the generalists. We would therefore recommend our client to open a Ramen joint to further avoid competition. 

**6 Discussion**  
While the results of our analysis might be a good starting point there is several things to consider. Firstly we had to exclude several neighborhoods from our analysis for a lack of data. In some areas we received information from the FourSquare API for a mere two venues. This might be because there is really few venues in that area or because the venues are missing on FourSquare or just because the 500 metre radius is too small. This leads to the second problem that the area in a 500 metre radius arount the centre of an area might not capture the area very well, either because it is smaller, bigger or just not very circular. Here a different method to retrieve venues might be preferable. Lastly we don't know if the existing venue structure really predicts the success of a new business very well. Demographic data for example might be more useful.

**7 Conclusion**  
Our recommendation to open a Ramen restaurant in the postal code area 40215 looks sound considering the underlying data. However, with more data sources and some refinements in the analysis we could gain more confidence in our advice.